<a href="https://colab.research.google.com/github/ALEJOOYT/ConvertirNotebookHTML/blob/main/ConvertirNotebookHTML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>Zalles Merino Izai Alejandro</center>

In [24]:
# Convertidor de Notebook a HTML Personalizable
# Desarrollado para Google Colab

# Importar bibliotecas necesarias
from google.colab import files
import subprocess
import IPython.display as display
from IPython.display import HTML
import os

In [ ]:
def MostrarMensaje(mensaje, tipo="info"):
    """Muestra mensajes formateados en la consola según su tipo."""
    if tipo == "info":
        print(f"ℹ️ {mensaje}")
    elif tipo == "exito":
        print(f"✅ {mensaje}")
    elif tipo == "error":
        print(f"❌ {mensaje}")
    elif tipo == "alerta":
        print(f"⚠️ {mensaje}")
    elif tipo == "proceso":
        print(f"⏳ {mensaje}")
    elif tipo == "descarga":
        print(f"⬇️ {mensaje}")
    elif tipo == "completado":
        print(f"✨ {mensaje}")

In [ ]:
def CrearMenuPersonalizado(titulo):
    """Crea un menú HTML estilizado para insertar en el notebook."""
    menuHTML = f"""
    <div style="background-color: #3498db; color: white; padding: 15px; margin-bottom: 20px; border-radius: 8px; box-shadow: 0 4px 8px rgba(0,0,0,0.2);">
        <h1 style="margin: 0; text-align: center; font-family: Arial, sans-serif;">{titulo}</h1>
        <div style="display: flex; justify-content: center; margin-top: 15px;">
            <div style="background-color: #2980b9; padding: 8px 15px; margin: 0 10px; border-radius: 5px; cursor: pointer; transition: background-color 0.3s; font-weight: bold;">Inicio</div>
            <div style="background-color: #2980b9; padding: 8px 15px; margin: 0 10px; border-radius: 5px; cursor: pointer; transition: background-color 0.3s; font-weight: bold;">Contenido</div>
            <div style="background-color: #2980b9; padding: 8px 15px; margin: 0 10px; border-radius: 5px; cursor: pointer; transition: background-color 0.3s; font-weight: bold;">Descargar</div>
        </div>
    </div>
    """
    return menuHTML

In [ ]:
def LeerArchivoConDeteccionCodificacion(rutaArchivo):
    """Lee un archivo probando diferentes codificaciones hasta encontrar la correcta."""
    # Lista de codificaciones a intentar
    listaCodificaciones = ['utf-8', 'latin-1', 'iso-8859-1', 'cp1252']

    for codificacion in listaCodificaciones:
        try:
            with open(rutaArchivo, 'r', encoding=codificacion) as archivo:
                contenido = archivo.read()
                MostrarMensaje(f"Archivo leído correctamente usando codificación: {codificacion}", "info")
                return contenido, codificacion
        except UnicodeDecodeError:
            continue

    # Si ninguna codificación específica funciona, usamos latin-1 como último recurso
    with open(rutaArchivo, 'r', encoding='latin-1') as archivo:
        contenido = archivo.read()
        MostrarMensaje("Se usó codificación latin-1 como último recurso", "alerta")
        return contenido, 'latin-1'

In [ ]:
def InsertarMenuEnHTML(contenidoHTML, menuHTML):
    """Inserta el menú personalizado en el contenido HTML."""
    if "<body>" in contenidoHTML:
        contenidoModificado = contenidoHTML.replace("<body>", f"<body>\n{menuHTML}")
        return contenidoModificado
    elif "</head>" in contenidoHTML:
        contenidoModificado = contenidoHTML.replace("</head>", f"</head>\n{menuHTML}")
        return contenidoModificado
    else:
        # Si no encuentra las etiquetas anteriores, lo inserta al inicio
        return f"{menuHTML}\n{contenidoHTML}"

In [ ]:
def GuardarArchivoHTML(rutaArchivo, contenido, codificacion):
    """Guarda el contenido en un archivo HTML con la codificación especificada."""
    try:
        with open(rutaArchivo, 'w', encoding=codificacion) as archivo:
            archivo.write(contenido)
        MostrarMensaje(f"Archivo guardado exitosamente: {rutaArchivo}", "exito")
        return True
    except Exception as error:
        MostrarMensaje(f"Error al guardar el archivo: {str(error)}", "error")
        return False

In [ ]:
def ConvertirNotebookAHTML():
    """Función principal que coordina todo el proceso de conversión."""
    # Mostrar mensaje de bienvenida
    print("\n" + "="*80)
    print("🔄 CONVERTIDOR DE NOTEBOOK A HTML".center(80))
    print("="*80 + "\n")

    # Paso 1: Solicitar al usuario que suba su archivo
    MostrarMensaje("Por favor, sube tu archivo de notebook (.ipynb):", "proceso")
    archivosSubidos = files.upload()

    if not archivosSubidos:
        MostrarMensaje("No se seleccionó ningún archivo. Proceso cancelado.", "error")
        return

    # Obtener nombre del archivo subido
    nombreArchivo = list(archivosSubidos.keys())[0]

    if not nombreArchivo.endswith('.ipynb'):
        MostrarMensaje(f"El archivo {nombreArchivo} no parece ser un notebook de Jupyter (.ipynb)", "alerta")
        respuesta = input("¿Deseas continuar de todos modos? (s/n): ").lower()
        if respuesta != 's':
            MostrarMensaje("Proceso cancelado por el usuario.", "info")
            return

    # Paso 2: Instalar nbconvert si es necesario
    MostrarMensaje("Verificando dependencias necesarias...", "proceso")
    subprocess.run(["pip", "install", "nbconvert"], capture_output=True)

    # Preguntar si desea incluir menú
    incluirMenu = False
    respuestaMenu = input("\n¿Deseas incluir un menú en la parte superior del HTML? (s/n): ").lower()
    if respuestaMenu == 's':
        incluirMenu = True

    # Paso 3: Convertir notebook a HTML
    MostrarMensaje(f"Convirtiendo el notebook {nombreArchivo} a formato HTML...", "proceso")
    resultado = subprocess.run(["jupyter", "nbconvert", nombreArchivo, "--to", "html"], capture_output=True)

    if resultado.returncode != 0:
        MostrarMensaje("Error al convertir el notebook a HTML.", "error")
        MostrarMensaje(f"Detalles del error: {resultado.stderr.decode('utf-8', errors='ignore')}", "error")
        return

    # Confirmar nombre del archivo HTML generado
    archivoHTML = nombreArchivo.replace(".ipynb", ".html")
    if not os.path.exists(archivoHTML):
        MostrarMensaje(f"No se encontró el archivo HTML generado: {archivoHTML}", "error")
        return

    # Paso 4: Leer el archivo HTML generado
    MostrarMensaje("Procesando el archivo HTML generado...", "proceso")
    try:
        contenidoHTML, codificacionUsada = LeerArchivoConDeteccionCodificacion(archivoHTML)
    except Exception as error:
        MostrarMensaje(f"Error al leer el archivo HTML: {str(error)}", "error")
        return

    # Paso 5: Si se solicita, crear e insertar el menú personalizado
    contenidoModificado = contenidoHTML

    if incluirMenu:
        tituloNotebook = nombreArchivo.replace(".ipynb", "").replace("_", " ").title()
        menuPersonalizado = CrearMenuPersonalizado(f"Notebook: {tituloNotebook}")
        contenidoModificado = InsertarMenuEnHTML(contenidoHTML, menuPersonalizado)
        MostrarMensaje("Menú personalizado insertado correctamente.", "exito")

    # Paso 6: Guardar el archivo HTML modificado
    exitoGuardado = GuardarArchivoHTML(archivoHTML, contenidoModificado, codificacionUsada)
    if not exitoGuardado:
        return

    # Paso 7: Descargar el archivo HTML modificado
    MostrarMensaje("Preparando archivo para descarga...", "descarga")
    files.download(archivoHTML)

    # Finalizar con mensaje de éxito
    print("\n" + "-"*80)
    MostrarMensaje("¡Proceso completado con éxito!", "completado")
    if incluirMenu:
        MostrarMensaje(f"El archivo '{archivoHTML}' ha sido generado y descargado con el menú personalizado", "completado")
    else:
        MostrarMensaje(f"El archivo '{archivoHTML}' ha sido generado y descargado sin modificaciones adicionales", "completado")
    print("-"*80 + "\n")

In [ ]:
# Ejecutar la función principal
ConvertirNotebookAHTML()